In [3]:
# Import Libraries
import requests
import pandas as pd
import time

In [4]:
# Keys
API_KEY = 'XXXXX'
CHANNEL_ID = 'XXXXX'

In [5]:
def get_video_details(video_id):
    video_url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(video_url).json()
    
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    return view_count, like_count, favorite_count, comment_count

In [6]:
def get_videos(df):
    pageToken = ""
    while True:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
        response = requests.get(url).json()
        time.sleep(1)  #give it a second before starting the for loop
        if 'items' in response:
            for video in response['items']:
                if video['id']['kind'] == 'youtube#video':
                    video_id = video['id']['videoId']
                    video_title = video['snippet']['title']
                    video_title = str(video_title).replace("&","")
                    upload_date = video['snippet']['publishedAt']
                    upload_date = str(upload_date).split("T")[0]
                    view_count, like_count, favorite_count, comment_count = get_video_details(video_id)

                    new_row = pd.DataFrame([[video_id, video_title, upload_date, view_count, like_count, favorite_count, comment_count]], columns=df.columns)
                    df = pd.concat([df, new_row], ignore_index=True)
                    # df = df.append(new_row, ignore_index=True)

                    # df = df.append({'video_id':video_id,'video_title':video_title,
                    #             "upload_date":upload_date,"view_count":view_count,
                    #             "like_count":like_count,"favorite_count":favorite_count,
                    #             "comment_count":comment_count}, ignore_index=True)
            try:
                if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                    pageToken = "pageToken=" + response['nextPageToken']

            except:
                break
        else:
            print("No 'items' found in response")
            print(response)  # Print the full response for debugging
            break
    return df

In [7]:
df1 = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'favorite_count', 'comment_count'])
df1 = get_videos(df1)


In [8]:
df1

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,CSmZKgzeYQM,Cute little chicks,2020-07-26,38,4,0,0


In [9]:
df1.to_csv("YoutubeVideoData.csv")